In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment, DataTypes, CsvTableSource
import datetime
from pyflink.table.expressions import col
from pyflink.table.window import Over, GroupWindow
from pyflink.table.expressions import col, UNBOUNDED_RANGE, CURRENT_RANGE
from pyflink.table.udf import udf
# create a batch TableEnvironment
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)
#table_env.get_config().get_configuration().set_string("pipeline.jars", "file:///opt/flink-1.15.0/lib/flink-sql-orc-1.15.0.jar")
#table_env.get_config().get_configuration().set_string("pipeline.jars", "file:///opt/flink-1.15.3/lib/flink-json-1.15.3.jar")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.3/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [2]:
# InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
# 536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 8:26,2.55,17850,United Kingdom
# here we use source connector called filesystem [reading files/folder,nfs,hdfs etc]
source_ddl = """
            CREATE TABLE invoices (
                   InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING,
                   WATERMARK FOR InvoiceDate AS InvoiceDate - INTERVAL '1' SECOND

                    ) WITH (
                      'connector' = 'filesystem',          
                     'path' = 'file:////home/training/flink-dev/data/ecommerce-clean.csv', 
                      'format' = 'csv'
                    )"""

# this will create flink table called invoices in default catalog and default db
table_env.execute_sql(source_ddl) 

invoices = table_env.from_path("invoices")
##############################
print('\nRegistered Tables List')
print(table_env.list_tables())

print('\nFinancial Trxs Schema')
invoices.print_schema()


Registered Tables List
['invoices']

Financial Trxs Schema
(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `InvoiceDate` TIMESTAMP(3),
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING,
  WATERMARK FOR `InvoiceDate`: TIMESTAMP(3) AS `InvoiceDate` - INTERVAL '1' SECOND
)


In [3]:
# format is json, json jar already present in lib directory, loaded into memory by default
sink_ddl_parquet=f"""
    create table `EComJsonResult`(
                 InvoiceNo STRING,
                   StockCode  STRING,
                   Description  STRING,
                   Quantity DOUBLE,
                   InvoiceDate TIMESTAMP(3),
                   UnitPrice DOUBLE,
                   CustomerID STRING,
                   Country STRING
    ) with (
        'connector' = 'filesystem',
        'format' = 'json',
         'path' = 'file:////home/training/flink-dev/data/ecommerce-clean.json'
    )
    """

table_env.execute_sql(sink_ddl_parquet)

In [4]:
table_env.execute_sql("INSERT INTO EComJsonResult select * from invoices").wait()
invoices.execute_insert("EComJsonResult")